In [2]:
%cd /Users/gwg/local/dmcm
%matplotlib inline

/Users/gwg/local/dmcm


In [3]:
import torch
import random
import matplotlib.pyplot as plt
from   matplotlib.offsetbox import OffsetImage, AnnotationBbox
from   matplotlib.patches import Ellipse
import seaborn

from   sklearn.decomposition import PCA
from   sklearn.manifold import TSNE

from   data import GTExConfig, GTExDataset

COLORS = [
    '#a6cee3', '#b15928', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c',
    '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#99995b', '#1f78b4'
]

In [4]:
cfg = GTExConfig()
cfg.PCCA_Z_DIM = 300
dataset = GTExDataset(cfg)

Z = torch.load('best_embeddings.pt', map_location='cpu')
print(Z.shape)
N = 2221

torch.Size([2221, 300])


In [ ]:
def imscatter(points, images, colors, ax, zoom=1, frameon=True):
    for (x, y), image in zip(points, images):
        try:
            image = np.asarray(Image.fromarray(image).convert('RGB'))
        except:
            pass
        im = OffsetImage(image.T, zoom=0.05)
        ab = AnnotationBbox(im, (x, y), xycoords='data', frameon=frameon)
        ax.add_artist(ab)

# ------------------------------------------------------------------------------

def plot(points, colors, images=False):
    fig, ax = plt.subplots(1)
    fig.set_size_inches(20, 20)

    Xp = points[:, 0]
    Yp = points[:, 1]
    
    ax.set_xlim([Xp.min(), Xp.max()])
    ax.set_ylim([Yp.min(), Yp.max()])
    
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    
    if images:
        images = dataset.images.detach().numpy()
        imscatter(points, images, colors, ax)
    else:
        for xp, yp, c in zip(Xp, Yp, colors):
            ax.scatter(xp, yp, c=c, s=10)
            
# ------------------------------------------------------------------------------

def plot_sbs(points, colors):
    fig, axes = plt.subplots(2)
    fig.set_size_inches(10, 20)
    
    Xp = points[:, 0]
    Yp = points[:, 1]
    
    ax1 = axes[0]
    ax1.set_xlim([Xp.min(), Xp.max()])
    ax1.set_ylim([Yp.min(), Yp.max()])
    
    clrs = seaborn.color_palette('hls', n_colors=len(dataset.classes))
    LINE_STYLES = ['o', 'v', 's', '*']
    NUM_STYLES = len(LINE_STYLES)

    for i in range(len(dataset.classes)):
        indices = dataset.labels[:N] == i
        x = Xp[indices]
        y = Yp[indices]
        label = dataset.labelEncoder.inverse_transform([i])[0]
        marker = LINE_STYLES[i % NUM_STYLES]
        ax1.scatter(x, y, s=1, c=clrs[i], label=label, marker=marker, zorder=10)

#     ax2 = axes[1]
#     ax2.set_xlim([Xp.min(), Xp.max()])
#     ax2.set_ylim([Yp.min(), Yp.max()])
#     images = dataset.images.detach().numpy()
#     imscatter(points, images, colors, ax2)

In [ ]:
L = dataset.labels[:N]
Z = Z[:N, :]

k = 100
Zc = Z[:, :k]
Za = Z[:, k:2*k]
Zb = Z[:, 2*k:]

pca = PCA(n_components=50)
tsne = TSNE(n_components=2)
Z_ = tsne.fit_transform(pca.fit_transform(Za))
# Z_ = pca.fit_transform(Zb)

In [ ]:
plot_sbs(Z_, L)